# RAG Evaluation
#### 작성: 고우주

RAG(Retrieval-Augmented Generation) 모델의 성능을 평가하는 주요 지표들을 계산하는 파이썬 코드 예시입니다. 실제 평가에서는 더 정교한 자연어 이해(NLU) 기술이나 대규모 언어 모델(LLM)을 평가자로 활용하는 방안도 고려할 수 있지만, 여기서는 일반적으로 사용되는 라이브러리를 활용하여 각 지표의 개념을 이해하고 직접 산출해볼 수 있는 코드를 제공합니다.

### 평가 시나리오 설정

먼저, 평가를 위한 가상의 데이터를 정의합니다.

  * **사용자 질문 (Question):** 사용자가 시스템에 입력한 질문
  * **원본 텍스트 (Retrieved Context):** RAG 모델이 검색 단계에서 가져온 원본 문서 또는 문맥
  * **생성된 답변 (Generated Answer):** RAG 모델이 검색된 문맥을 기반으로 최종 생성한 답변
  * **정답 (Ground Truth Answer):** 사람이 직접 작성한 이상적인 답변

## 평가 가상 데이터

In [18]:
# 사용자의 질문
question = "대한민국의 수도는 어디인가요?"

# RAG 모델이 검색한 문맥 (Retrieved Context)
# 이 중 첫 번째, 두 번째 문장이 답변 생성에 직접적으로 기여했다고 가정
retrieved_context = """
대한민국은 동아시아에 위치한 국가이다.
수도는 서울특별시이며, 대한민국의 정치, 경제, 사회, 문화의 중심지 역할을 한다.
서울의 인구는 약 940만 명이다.
부산은 대한민국 제2의 도시이자 최대 항구 도시이다.
"""

# RAG 모델이 생성한 답변 (Generated Answer)
generated_answer = "대한민국의 수도는 서울입니다."

# 정답 (Ground Truth)
ground_truth_answer = "대한민국의 수도는 서울특별시입니다."

## 평가 코드

In [19]:
#%pip install -q sentence-transformers scikit-learn numpy

In [20]:
# 필요한 라이브러리를 임포트
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import re

## 1. 평가 시나리오 설정

In [21]:
# 사용자의 질문
question = "대한민국의 수도는 어디인가요?"

# RAG 모델이 검색한 문맥 (Retrieved Context)
# 가정: 이 중 첫 번째, 두 번째 문장이 답변 생성에 직접적으로 기여
retrieved_context = """
대한민국은 동아시아에 위치한 국가이다.
수도는 서울특별시이며, 대한민국의 정치, 경제, 사회, 문화의 중심지 역할을 한다.
서울의 인구는 약 940만 명이다.
부산은 대한민국 제2의 도시이자 최대 항구 도시이다.
"""
# 문맥을 문장 단위로 분리합
context_sentences = [s.strip() for s in retrieved_context.strip().split('\n')]

# RAG 모델이 생성한 답변
generated_answer = "대한민국의 수도는 서울입니다."

# 정답 (Ground Truth)
ground_truth_answer = "대한민국의 수도는 서울특별시입니다."

# 답변 생성에 실제로 필요한 이상적인 문맥(가정)
# Context Recall 계산에 사용됩니다.
ideal_context = "대한민국의 수도는 서울특별시이다."

print(f"# 사용자 질문: {question}")
print(f"# 검색된 문맥: {retrieved_context.strip()}")
print(f"# 생성된 답변: {generated_answer}")
print(f"# 정답: {ground_truth_answer}\n")

# 사용자 질문: 대한민국의 수도는 어디인가요?
# 검색된 문맥: 대한민국은 동아시아에 위치한 국가이다.
수도는 서울특별시이며, 대한민국의 정치, 경제, 사회, 문화의 중심지 역할을 한다.
서울의 인구는 약 940만 명이다.
부산은 대한민국 제2의 도시이자 최대 항구 도시이다.
# 생성된 답변: 대한민국의 수도는 서울입니다.
# 정답: 대한민국의 수도는 서울특별시입니다.



## 2. 모델 로드
- 한국어 문장을 임베딩하기 위해 사전 훈련된 모델을 로드합니다.
- 'ko-sroberta-multitask'는 한국어 문장의 의미를 잘 포착하는 모델 중 하나

In [22]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def get_word_tokens(text):
    """
    간단한 토크나이저: 텍스트에서 단어(한글, 영어, 숫자)를 추출합니다.
    """
    return set(re.findall(r'[\w\d]+', text.lower()))

## 3. 평가 지표

### 1. Faithfulness (충실성)
- 생성된 답변이 검색된 문맥에 얼마나 충실한지를 평가합니다.
- 생성된 답변과 문맥 간의 의미적 유사도를 측정합니다.

In [33]:
answer_embedding = model.encode(generated_answer, convert_to_tensor=True)
context_embedding = model.encode(retrieved_context, convert_to_tensor=True)
faithfulness_score = util.pytorch_cos_sim(answer_embedding, context_embedding).item()

print(f"Faithfulness: {faithfulness_score:.4f}\n")

Faithfulness: 0.6679



### 2. Answer Relevancy (답변 관련성)
- 생성된 답변이 사용자의 질문에 얼마나 관련성이 있는지를 평가합니다.
- 질문과 생성된 답변 간의 의미적 유사도를 측정합니다.

In [34]:
question_embedding = model.encode(question, convert_to_tensor=True)
answer_embedding = model.encode(generated_answer, convert_to_tensor=True)
answer_relevancy_score = util.pytorch_cos_sim(question_embedding, answer_embedding).item()

print(f"Answer Relevancy: {answer_relevancy_score:.4f}\n")

Answer Relevancy: 0.6988



### 3. Context Relevancy (문맥 관련성)
- 검색된 문맥이 사용자의 질문과 얼마나 관련성이 있는지를 평가합니다.
- 질문과 검색된 각 문장 간의 평균 유사도를 계산합니다.

In [35]:
question_embedding = model.encode(question, convert_to_tensor=True)
context_sentence_embeddings = model.encode(context_sentences, convert_to_tensor=True)
relevancy_scores = [util.pytorch_cos_sim(question_embedding, s_emb).item() for s_emb in context_sentence_embeddings]
context_relevancy_score = np.mean(relevancy_scores)

print(f"Context Relevancy: {context_relevancy_score:.4f}\n")

Context Relevancy: 0.4563



### 4. Context Precision (문맥 정확성)
- 검색된 문맥 중 실제로 답변 생성에 기여한 문장의 비율을 나타냅니다.
- 여기서는 답변과 각 문맥 문장 간의 유사도가 특정 임계값(예: 0.5)을 넘으면 기여했다고 가정합니다.

In [36]:
answer_embedding = model.encode(generated_answer, convert_to_tensor=True)
context_sentence_embeddings = model.encode(context_sentences, convert_to_tensor=True)
precision_scores = [util.pytorch_cos_sim(answer_embedding, s_emb).item() for s_emb in context_sentence_embeddings]

# 유사도가 0.5 이상인 문장을 '사용한' 문장으로 간주
used_sentences = sum(1 for score in precision_scores if score > 0.5)
context_precision_score = used_sentences / len(context_sentences) if len(context_sentences) > 0 else 0

print(f"Context Precision: {context_precision_score:.4f} ({used_sentences} / {len(context_sentences)})\n")

Context Precision: 0.5000 (2 / 4)



### 5. Context Recall (문맥 재현율)
- 답변 생성에 필요한 전체 정보 중 RAG가 얼마나 찾아냈는지를 평가합니다.
- 여기서는 'ideal_context'에 필요한 정보가 모두 담겨있다고 가정하고,
- 검색된 문맥(retrieved_context)이 이 정보를 얼마나 포함하는지 단어 단위로 확인합니다.

In [37]:
ideal_tokens = get_word_tokens(ideal_context)
retrieved_tokens = get_word_tokens(retrieved_context)
retrieved_ideal_tokens = ideal_tokens.intersection(retrieved_tokens)
context_recall_score = len(retrieved_ideal_tokens) / len(ideal_tokens) if len(ideal_tokens) > 0 else 0

print(f"Context Recall: {context_recall_score:.4f}\n")

Context Recall: 0.6667



### 6. Answer Semantic Similarity (답변 의미 유사도)
- 생성된 답변이 정답(Ground Truth)과 의미적으로 얼마나 유사한지를 평가합니다.

In [38]:
answer_embedding = model.encode(generated_answer, convert_to_tensor=True)
ground_truth_embedding = model.encode(ground_truth_answer, convert_to_tensor=True)
semantic_similarity_score = util.pytorch_cos_sim(answer_embedding, ground_truth_embedding).item()

print(f"Answer Semantic Similarity: {semantic_similarity_score:.4f}\n")

Answer Semantic Similarity: 0.8835



### 7. Answer Correctness (답변 정확성)
- 생성된 답변이 사실적으로 올바른지를 평가합니다.
- 여기서는 정답에 포함된 핵심 단어가 생성된 답변에도 포함되어 있는지로 간단히 확인합니다.

In [ ]:
#%pip install konlpy

In [41]:
# 형태소 분석기 로드
from konlpy.tag import Okt
okt = Okt()

def get_nouns(text):
    """
    형태소 분석기를 사용하여 텍스트에서 명사만 추출
    """
    return set(okt.nouns(text))

# 형태소 분석 기반의 Answer Correctness 계산 로직
generated_answer = "대한민국의 수도는 서울입니다."
key_nouns = {'대한민국', '수도', '서울'}

# 생성된 답변에서 명사 추출
generated_answer_nouns = get_nouns(generated_answer)

contained_key_nouns = key_nouns.intersection(generated_answer_nouns)
correctness_score = len(contained_key_nouns) / len(key_nouns) if len(key_nouns) > 0 else 0

print(f"Answer Correctness: {correctness_score:.4f} (핵심 명사 {len(key_nouns)}개 중 {len(contained_key_nouns)}개 포함)\n")

Answer Correctness: 1.0000 (핵심 명사 3개 중 3개 포함)

